In [54]:
with open("data/blog.txt", 'r', encoding='utf-8') as f:
    text = f.read()
tokens = list(map(int,text.encode("utf-8")))

In [55]:
print(tokens[:20])
len(tokens)

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189]


24213

In [56]:
def get_stat(ids):
    count = {}
    for pair in zip(ids,ids[1:]):
        count[pair] = count.get(pair,0) + 1
    return count

def merge(ids, pair, idx):
    new_ids = []
    i=0
    while i< len(ids):
        if i< len(ids)-1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i+=2
        else:
            new_ids.append(ids[i])
            i+=1
    return new_ids


In [57]:
vocab_size = 276
num_merges = vocab_size - 256

ids = list(tokens)

merges = {}
for i in range(num_merges):
    counts = get_stat(ids)
    pair = max(counts, key=counts.get)
    idx = 256 + i
    ids = merge(ids,pair,idx)
    merges[pair] = idx

In [79]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0]+vocab[p1]

def decode(ids:list[int])->str:
    tokens = b"".join([vocab[idx] for idx in ids])
    text = tokens.decode('utf-8', errors='replace')
    return text

def encode(text:str)-> list[int]:
    tokens = list(text.encode("utf-8"))
    while len(tokens) > 1:
        stats = get_stat(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))
        if pair not in merges:
            break # there is not to merge
        idx = merges[pair]
        tokens = merge(tokens,pair,idx)

    return tokens
    

In [80]:
text[:10]

'Ｕｎｉｃｏｄｅ! 🅤'

In [83]:
decode(encode(text[:10]))

'Ｕｎｉｃｏｄｅ! 🅤'